In [1]:
import pandas as pd

## Get Information from Metadata
https://www.synapse.org/Synapse:syn50944327

In [2]:
# Read in the data
biospecimen = pd.read_csv("UCI_BIN1_biospecimen_metadata.csv")
individual = pd.read_csv("UCI_BIN1_individualID_metadata.csv")
rnaseqmetadata = pd.read_csv("UCI_BIN1_scRNAseq_assay.csv")

# Drop empty rows (all-NaN) for each dataframe
biospecimen = biospecimen.dropna(how='all')
individual = individual.dropna(how='all')
rnaseqmetadata = rnaseqmetadata.dropna(how='all')


# Print basic info for each dataframe
for name, df in [("biospecimen", biospecimen), ("individual", individual), ("rnaseqmetadata", rnaseqmetadata)]:
    print(f"\n=== {name.upper()} DATAFRAME ===")
    print(f"Shape: {df.shape[0]} rows x {df.shape[1]} columns")
    print(f"Columns: {list(df.columns)}")
    if 'individualID' in df.columns:
        print(f"Number of unique individualIDs: {df['individualID'].nunique()}")
        print(f"Number of rows with missing individualID: {df['individualID'].isna().sum()}")
    else:
        print("'individualID' column not found.")



=== BIOSPECIMEN DATAFRAME ===
Shape: 653 rows x 17 columns
Columns: ['individualID', 'specimenID', 'specimenIdSource', 'organ', 'tissue', 'BrodmannArea', 'sampleStatus', 'tissueWeight', 'tissueVolume', 'nucleicAcidSource', 'cellType', 'fastingState', 'isPostMortem', 'samplingAge', 'samplingAgeUnits', 'visitNumber', 'assay']
Number of unique individualIDs: 112
Number of rows with missing individualID: 0

=== INDIVIDUAL DATAFRAME ===
Shape: 112 rows x 27 columns
Columns: ['individualID', 'climbID', 'microchipID', 'birthID', 'matingID', 'individualIdSource', 'materialOrigin', 'sex', 'species', 'generation', 'dateBirth', 'ageDeath', 'ageDeathUnits', 'brainWeight', 'rodentWeight', 'rodentDiet', 'bedding', 'room', 'waterpH', 'treatmentDose', 'treatmentType', 'stockNumber', 'genotype', 'genotypeBackground', 'individualCommonGenotype', 'modelSystemName', 'officialName']
Number of unique individualIDs: 112
Number of rows with missing individualID: 0

=== RNASEQMETADATA DATAFRAME ===
Shape: 8 r

In [3]:
# Merge the dataframes
merged1_df = pd.merge(biospecimen, individual, on='individualID', how='outer')
merged_df = pd.merge(merged1_df, rnaseqmetadata, on='specimenID', how='outer')

print("\n=== MERGED DATAFRAME INFO ===")
print(f"Shape: {merged_df.shape[0]} rows x {merged_df.shape[1]} columns")
print(f"Columns: {list(merged_df.columns)}")

if 'individualID' in merged_df.columns:
    print(f"Number of unique individualIDs: {merged_df['individualID'].nunique()}")
    print(f"Number of rows with missing individualID: {merged_df['individualID'].isna().sum()}")
else:
    print("'individualID' column not found in merged_df.")

if 'specimenID' in merged_df.columns:
    print(f"Number of unique specimenIDs: {merged_df['specimenID'].nunique()}")
    print(f"Number of rows with missing specimenID: {merged_df['specimenID'].isna().sum()}")
else:
    print("'specimenID' column not found in merged_df.")

print("\nFirst 5 rows of merged_df:")
print(merged_df.head())



=== MERGED DATAFRAME INFO ===
Shape: 653 rows x 62 columns
Columns: ['individualID', 'specimenID', 'specimenIdSource', 'organ', 'tissue', 'BrodmannArea', 'sampleStatus', 'tissueWeight', 'tissueVolume', 'nucleicAcidSource', 'cellType', 'fastingState', 'isPostMortem', 'samplingAge', 'samplingAgeUnits', 'visitNumber', 'assay_x', 'climbID', 'microchipID', 'birthID', 'matingID', 'individualIdSource', 'materialOrigin', 'sex', 'species', 'generation', 'dateBirth', 'ageDeath', 'ageDeathUnits', 'brainWeight', 'rodentWeight', 'rodentDiet', 'bedding', 'room', 'waterpH', 'treatmentDose', 'treatmentType', 'stockNumber', 'genotype', 'genotypeBackground', 'individualCommonGenotype', 'modelSystemName', 'officialName', 'assay_y', 'platform', 'RIN', 'rnaBatch', 'libraryBatch', 'sequencingBatch', 'libraryPrep', 'libraryPreparationMethod', 'libraryType', 'sampleBarcode', 'isStranded', 'readStrandOrigin', 'readLength', 'runType', 'totalReads', 'validBarcodeReads', 'numberCells', 'medianGenes', 'medianUMIs

In [6]:
# Just looking at some stuff, not neccessary to run
# Print unique entries for the "tissue" column in merged_df
if "tissue" in merged_df.columns:
    unique_tissues = merged_df["tissue"].dropna().unique()
    print("\nUnique entries in 'tissue' column:")
    for t in unique_tissues:
        print(f"- {t}")
    print(f"\nTotal unique tissues: {len(unique_tissues)}")
else:
    print("'tissue' column not found in merged_df.")

# Print unique entries for the "genotype" column in merged_df
if "genotype" in merged_df.columns:
    unique_genotypes = merged_df["genotype"].dropna().unique()
    print("\nUnique entries in 'genotype' column:")
    for g in unique_genotypes:
        print(f"- {g}")
    print(f"\nTotal unique genotypes: {len(unique_genotypes)}")
else:
    print("'genotype' column not found in merged_df.")

# Print the number of entries for each tissue for each genotype
if "genotype" in merged_df.columns and "tissue" in merged_df.columns:
    counts = merged_df.groupby(["genotype", "tissue"]).size().reset_index(name="count")
    print("\nNumber of entries for each tissue for each genotype:")
    if counts.empty:
        print("No data found for genotype and tissue combinations.")
    else:
        for g in counts["genotype"].unique():
            print(f"- {g}:")
            sub = counts[counts["genotype"] == g]
            for _, row in sub.iterrows():
                print(f"    - {row['tissue']}: {row['count']}")
else:
    print("Either 'genotype' or 'tissue' column not found in merged_df.")

# Print the number of unique (specimenID, individualID) pairs in merged_df
if 'specimenID' in merged_df.columns and 'individualID' in merged_df.columns:
    unique_pairs = merged_df[['specimenID', 'individualID']].drop_duplicates()
    print(f"\nNumber of unique (specimenID, individualID) pairs: {len(unique_pairs)}")
else:
    print("Either 'specimenID' or 'individualID' column not found in merged_df.")





Unique entries in 'tissue' column:
- plasma
- cerebral cortex
- hippocampus

Total unique tissues: 3

Unique entries in 'genotype' column:
- 5XFAD_carrier, BIN1-K358R_homozygous
- 5XFAD_carrier
- BIN1-K358R_homozygous
- 5XFAD_noncarrier
- AppAbeta_LoxP_noncarrier

Total unique genotypes: 5

Number of entries for each tissue for each genotype:
- 5XFAD_carrier:
    - cerebral cortex: 65
    - hippocampus: 84
    - plasma: 21
- 5XFAD_carrier, BIN1-K358R_homozygous:
    - cerebral cortex: 71
    - hippocampus: 85
    - plasma: 22
- 5XFAD_noncarrier:
    - cerebral cortex: 71
    - hippocampus: 38
    - plasma: 21
- AppAbeta_LoxP_noncarrier:
    - hippocampus: 22
- BIN1-K358R_homozygous:
    - cerebral cortex: 74
    - hippocampus: 57
    - plasma: 22

Number of unique (specimenID, individualID) pairs: 653


In [5]:
# GET THE DATA FOR THE SPREADSHEET (https://docs.google.com/spreadsheets/d/11vmntFrno9ubNMOE8bbMejEROPygmeJZ/edit?pli=1&gid=757405125#gid=757405125)
# Group merged_df by "ageDeath", "sex", and "genotype" and count the number of rows in each group
grouped = merged_df.groupby(["ageDeath", "sex", "genotype"]).size().reset_index(name='count')

print("\n=== GROUPED COUNTS BY ageDeath, sex, genotype ===")
print(grouped)

print(f"\nNumber of unique groups: {grouped.shape[0]}")



=== GROUPED COUNTS BY ageDeath, sex, genotype ===
    ageDeath     sex                              genotype  count
0          4  female                         5XFAD_carrier     38
1          4  female  5XFAD_carrier, BIN1-K358R_homozygous     42
2          4  female                      5XFAD_noncarrier     14
3          4  female                 BIN1-K358R_homozygous     21
4          4    male                         5XFAD_carrier     42
5          4    male  5XFAD_carrier, BIN1-K358R_homozygous     42
6          4    male                      5XFAD_noncarrier     22
7          4    male                 BIN1-K358R_homozygous     22
8         12  female                         5XFAD_carrier     49
9         12  female  5XFAD_carrier, BIN1-K358R_homozygous     49
10        12  female                      5XFAD_noncarrier     49
11        12  female              AppAbeta_LoxP_noncarrier     12
12        12  female                 BIN1-K358R_homozygous     53
13        12    male     

## Validate Metadata
Look at the actual RNAseq data and make sure that all samples exist and align with what is stated in the metadata.
https://www.synapse.org/Synapse:syn50944329

In [6]:
# In terminal, run:
# synapse list syn50944329 > UCI_Bin1K358R_rnaseqdata_list.csv

df = pd.read_csv("UCI_Bin1K358R_rnaseqdata_list.csv", header=None, names=["syn_id", "file_name"])

In [8]:
# Parse the filename to get the metadata
def parse_filename(fname):
    # Split by underscore
    parts = str(fname).split("_")
    # Defensive: if not enough parts, return Nones
    if len(parts) < 7:
        return {
            "lane": None,
            "tissue": None,
            "sex": None,
            "genotype": None,
            "age": None,
            "individualID": None,
            "read": None
        }
    # Find the index of the sex (first "M" or "F")
    try:
        sex_idx = next(i for i, p in enumerate(parts) if p in ("M", "F"))
    except StopIteration:
        sex_idx = 1  # fallback, but may be wrong
    # Find the index of the age (endswith "mo")
    try:
        age_idx = next(i for i, p in enumerate(parts) if p.endswith("mo"))
    except StopIteration:
        age_idx = sex_idx + 2  # fallback
    # Find the index of the individualID (should be after lane)
    try:
        indiv_idx = age_idx + 2
        individualID = parts[indiv_idx]
    except IndexError:
        individualID = None
    # Find the index of the lane (should be after age)
    try:
        lane_idx = age_idx + 1
        lane = parts[lane_idx]
    except IndexError:
        lane = None
    # Read is always the last part before extension, e.g. ..._1.fq.gz or ..._2.fq.gz
    read_part = parts[-1]
    # If read_part contains a dot, split and take the first part (e.g. "1.fq.gz" -> "1")
    read = read_part.split(".")[0]
    # Genotype is everything between sex and age
    genotype = "_".join(parts[sex_idx + 1:age_idx])
    return {
        "lane": lane,
        "tissue": parts[0],
        "sex": parts[sex_idx],
        "genotype": genotype,
        "age": parts[age_idx],
        "individualID": individualID,
        "read": read
    }

parsed = df["file_name"].apply(parse_filename)
parsed_df = pd.DataFrame(parsed.tolist())
print(parsed_df)

    lane tissue sex   genotype   age individualID read
0     B1   hipp   F  5XFADHEMI  12mo        12433   R1
1     B1   hipp   F  5XFADHEMI  12mo        12433   R2
2     B1   hipp   F  5XFADHEMI  12mo        12440   R1
3     B1   hipp   F  5XFADHEMI  12mo        12440   R2
4     B1   hipp   F  5XFADHEMI  12mo        12450   R1
..   ...    ...  ..        ...   ...          ...  ...
143   B1   hipp   M    Bin1_HO   4mo        13037   R2
144   B1   hipp   M    Bin1_HO   4mo        13041   R1
145   B1   hipp   M    Bin1_HO   4mo        13041   R2
146   B1   hipp   M    Bin1_HO   4mo        13045   R1
147   B1   hipp   M    Bin1_HO   4mo        13045   R2

[148 rows x 7 columns]


In [10]:
# Make sure all samples have both read1 and read2
# For each unique sample (excluding 'read'), check if both read=1 and read=2 exist
group_cols = [col for col in parsed_df.columns if col != "read"]
read_counts = parsed_df.groupby(group_cols)["read"].nunique().reset_index()
# Entries where the number of unique reads is not 2 (i.e., missing a read)
missing_reads = read_counts[read_counts["read"] != 2]
if not missing_reads.empty:
    print("Entries missing either read=1 or read=2:")
    display(missing_reads)
else:
    print("All entries have both read=1 and read=2.")

All entries have both read=1 and read=2.


In [11]:
# Drop read 2
# Merge parsed_df back to df to keep all columns
merged_df_validation = pd.concat([df.reset_index(drop=True), parsed_df.reset_index(drop=True)], axis=1)

# Identify columns to compare for "exact same value" (excluding 'read')
compare_cols = [col for col in parsed_df.columns if col != "read"]

# Sort so that read=1 comes before read=2 for duplicates
merged_df_sorted = merged_df_validation.sort_values(by=compare_cols + ["read"])

# Drop duplicates, keeping the first (which will be read=1 if both exist)
dedup_df = merged_df_sorted.drop_duplicates(subset=compare_cols, keep="first")

new_df = dedup_df[parsed_df.columns].reset_index(drop=True)

In [12]:
# Get metadata to compare
# Group merged_df by "age", "sex", and "genotype" and count the number of rows in each group
grouped = new_df.groupby(["age", "sex", "genotype"]).size().reset_index(name='count')

print("\n=== GROUPED COUNTS BY age, sex, genotype ===")
print(grouped)

print(f"\nNumber of unique groups: {grouped.shape[0]}")


=== GROUPED COUNTS BY age, sex, genotype ===
     age sex           genotype  count
0   12mo   F          5XFADHEMI      5
1   12mo   F  5xFADHEMI_Bin1_HO      5
2   12mo   F            5xFADWT      5
3   12mo   F             BIN1HO      1
4   12mo   F            Bin1_HO      4
5   12mo   M          5XFADHEMI      4
6   12mo   M  5xFADHEMI_Bin1_HO      5
7   12mo   M            5xFADWT      5
8   12mo   M            Bin1_HO      5
9    4mo   F          5xFADHEMI      5
10   4mo   F  5xFADHEMI_Bin1_HO      4
11   4mo   F            5xFADWT      2
12   4mo   F            Bin1_HO      4
13   4mo   M          5xFADHEMI      5
14   4mo   M  5xFADHEMI_Bin1_HO      5
15   4mo   M            5xFADWT      5
16   4mo   M            Bin1_HO      5

Number of unique groups: 17
